<a href="https://colab.research.google.com/github/Mineth03/Dementia-Prediction-System-With-Comprehensive-Features/blob/Basic_Dementia_Prediction/Dementia%20prediction%20with%20common%20factors/Python%20Notebook/FacialRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Load the pretrained model
model_path = 'model.h5'  # Adjust the path
model = load_model(model_path)

# Load OpenCV's pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Depression weight mapping
depression_weights = {
    'Angry': 0.2,
    'Fear': 0.3,
    'Sad': 0.5,
    'Happy': -0.4,
    'Surprise': -0.1,
    'Disgust': 0.0,  # Optional, not strongly linked
    'Neutral': 0.0   # Neutral emotions ignored
}

# Function to detect faces in an image
def detect_face(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        return None, "Error: No face detected in the image."

    return gray, None

# Function to preprocess the input image
def preprocess_image(image):
    image = cv2.resize(image, (48, 48))  # Resize to model's input shape
    image = image / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    return image

# Function to calculate depression risk score
def calculate_depression_risk(predictions):
    drs = sum(depression_weights[label] * score for label, score in zip(emotion_labels, predictions))
    drs = max(0, min(drs, 1))  # Ensure value is between 0 and 1

    # Classify depression risk
    if drs < 0.3:
        risk_level = "Low Depression Risk"
    elif drs < 0.6:
        risk_level = "Moderate Depression Risk"
    else:
        risk_level = "High Depression Risk"

    return drs, risk_level

# Example usage
image_path = 'test.jpg'
gray_image, error_msg = detect_face(image_path)

if error_msg:
    print(error_msg)
else:
    processed_image = preprocess_image(gray_image)
    predictions = model.predict(processed_image)[0]  # Get prediction scores

    # Display scores
    for label, score in zip(emotion_labels, predictions):
        print(f"{label}: {score:.4f}")

    # Compute and display depression level
    depression_score, risk_category = calculate_depression_risk(predictions)
    print(f"\nDepression Risk Score: {depression_score:.4f}")
    print(f"Risk Category: {risk_category}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 597ms/step
Angry: 0.1350
Disgust: 0.0159
Fear: 0.1608
Happy: 0.3072
Sad: 0.1515
Surprise: 0.1746
Neutral: 0.0550

Depression Risk Score: 0.0107
Risk Category: Low Depression Risk
